In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Cafe.vn URL
url = "https://cafef.vn/du-lieu/tin-doanh-nghiep/fpt/event.chn"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"}

# Request the page
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find the correct news section  
news_section = soup.find("div", id = "divTopEvents") # Locate the div containing news

# Extract headlines, dates, and links
news_data = []
if news_section:
    items = news_section.find_all("li") # Each news item is inside <li>
    for item in items:
        title_tag = item.find("a") # Find the news title
        date_tag = item.find("span", class_ = "timeTitle") # Find the date

        if title_tag and date_tag:
            title = title_tag.text.strip()
            date = date_tag.text.strip()
            link = "https://cafef.vn" + title_tag["href"] # Full link

            news_data.append([date, title, link])

# Save to CSV
cafe = pd.DataFrame(news_data, columns = ["date", "headline", "link"])
cafe.to_csv("cafe.csv", index = False)

print ("Cafef.vn News Scarpped & Saved!")

Cafef.vn News Scarpped & Saved!


In [3]:
cafe = pd.read_csv("cafe.csv")

cafe.head()

,date,headline,link
0,01/03/2025 00:05,"Sau Vĩnh Phúc, Khánh Hòa, Bà Rịa - Vũng Tàu, F...",https://cafef.vn/sau-vinh-phuc-khanh-hoa-ba-ri...
1,27/02/2025 11:32,"Thủ tướng triệu tập lãnh đạo T&T, Hoà Phát, Ge...",https://cafef.vn/thu-tuong-trieu-tap-lanh-dao-...
2,26/02/2025 09:32,FPT bắt tay cùng “ông lớn” Ấn Độ đào tạo kỹ sư...,https://cafef.vn/fpt-bat-tay-cung-ong-lon-an-d...
3,25/02/2025 15:31,FPT mở chương trình đào tạo nhân lực ngành ô t...,https://cafef.vn/fpt-mo-chuong-trinh-dao-tao-n...
4,25/02/2025 11:57,FPT 'chơi lớn' tại Nhật Bản: Mua thêm 1 tòa nh...,https://cafef.vn/fpt-choi-lon-tai-nhat-ban-mua...


Vietstock

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Set up Selenium WebDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (no visible browser)
options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [5]:
# Start WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Open the Vietstock.vn news page
url = "https://finance.vietstock.vn/FPT/tin-tuc-su-kien.htm"
driver.get(url)

# Wait for JavaScript to load the content
time.sleep(5)  # Increase if necessary

# Extract news items
news_data = []
articles = driver.find_elements(By.CSS_SELECTOR, "a.text-link")  # Locate all news links

In [6]:
for article in articles:
    spans = article.find_elements(By.TAG_NAME, "span")  # Find all <span> inside <a>

    if len(spans) >= 2:
        title = spans[0].text.strip()  # First span = Headline
        date = spans[1].text.strip().replace("(", "").replace(")", "")  # Second span = Date
    else:
        title = article.text.strip()
        date = "Unknown"

    link = article.get_attribute("href")
    if link is None:
        link = "N/A"
    elif not link.startswith("http"):
        link = "https://vietstock.vn" + link  # Ensure full URL

    news_data.append([date, title, link])

In [7]:
# Close the browser
driver.quit()

# Save data to CSV
vietstock = pd.DataFrame(news_data, columns = ["date", "headline", "link"])
vietstock.to_csv("VietStock.csv", index = False)

print(vietstock.head())

         date                                           headline  \
0  04/03/2025  Phân tích kỹ thuật phiên chiều 04/03: Áp lực b...   
1  27/02/2025     FPT Telecom chi gần ngàn tỷ tạm ứng cổ tức 20%   
2  25/02/2025  Cuộc đua lợi nhuận VN30: Vinhomes giữ vững ngô...   
3  22/02/2025  VN-Index tiến sát 1,300 điểm, cổ phiếu nào đan...   
4  18/02/2025  Theo dấu dòng tiền cá mập 18/02: Tự doanh tiếp...   

                                                link  
0  https://vietstock.vn/2025/03/phan-tich-ky-thua...  
1  https://vietstock.vn/2025/02/fpt-telecom-chi-g...  
2  https://vietstock.vn/2025/02/cuoc-dua-loi-nhua...  
3  https://vietstock.vn/2025/02/vn-index-tien-sat...  
4  https://vietstock.vn/2025/02/theo-dau-dong-tie...  


In [9]:
# Load datasets
vietstock_df = pd.read_csv("VietStock.csv")
cafe = pd.read_csv("cafe.csv")

# Merge both datasets
merge = pd.concat([vietstock_df, cafe], ignore_index = False)

# Convert date columns to datetime format for sorting
merge["date"] = pd.to_datetime(merge["date"], format = "%d/%m/%Y", errors = "coerce")

# Sort by date (latest first)
merge = merge.sort_values(by = "date", ascending = False)

merge.to_csv("News.csv", index = False)

print(merge)

         date                                           headline  \
0  2025-03-04  Phân tích kỹ thuật phiên chiều 04/03: Áp lực b...   
30 2025-03-04  Quỹ đầu tư của "cá mập" Shark Tank muốn xây tr...   
33 2025-03-04  Chuyên gia chỉ rõ sự thật về đồng Pi, không hi...   
32 2025-03-04  Ông Trump sẽ áp thuế nông sản, kêu gọi nông dâ...   
31 2025-03-04  Kịch bản thao túng tâm lý, dẫn dụ con mồi tự n...   
..        ...                                                ...   
25        NaT  Chủ tịch FPT Trương Gia Bình hát Tái sinh giữa...   
26        NaT  Cơn bão DeepSeek càn quét cổ phiếu công nghệ: ...   
27        NaT   FPT: Báo cáo tình hình quản trị công ty năm 2024   
28        NaT  Doanh nhân Đỗ Cao Bảo kể về hành trình trở thà...   
29        NaT  Tổng giám đốc FPT: 10 năm chuẩn bị cho giấc mơ...   

                                                 link  
0   https://vietstock.vn/2025/03/phan-tich-ky-thua...  
30  https://vietstock.vn/2025/03/quy-dau-tu-cua-ca...  
33  https://vie

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyvi import ViTokenizer, ViPosTagger

In [12]:
df = merge

In [14]:
analyzer = SentimentIntensityAnalyzer()

In [15]:
def get_sentiment_vader(text):
    score = analyzer.polarity_scores(text)["compound"]
    return "Positive" if score > 0.05 else "Negative" if score < -0.05 else "Neutral"

def preprocess_vietnamese(text):
    return ViTokenizer.tokenize(text)

In [16]:
df["headline"] = df["headline"].astype(str)
df["headline_processed"] = df["headline"].apply(preprocess_vietnamese)
df["sentiment"] = df["headline_processed"].apply(get_sentiment_vader)

In [21]:
df.to_csv("news_sentiment.csv", encoding = "utf-8", index = False)

In [27]:
print(df["sentiment"].unique())

['Neutral' 'Positive']


In [ ]:
stock = pd.read_csv("FPT Corp Stock Price History.csv")